In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import sys
import requests
import urllib
import time

In [2]:
# Use the following site to get list of all publicly traded Stocks
#http://nxcoreapi.com/symbols.php

#nyse
nyse = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=NYSE&m_type=&s_type=exact&m_symbol=on&m_name=on')
#nasdaq
nqnm = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=NQNM&m_type=&s_type=exact&m_symbol=on&m_name=on')
#amex
amex = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=AMEX&m_type=&s_type=exact&m_symbol=on&m_name=on')

In [3]:
nyse = nyse[0]
nyse.columns = nyse.iloc[0]
nyse = nyse[1:]
# nyse

In [4]:
nqnm = nqnm[0]
nqnm.columns = nqnm.iloc[0]
nqnm = nqnm[1:]
# nqnm

In [5]:
amex = amex[0]
amex.columns = amex.iloc[0]
amex = amex[1:]
# amex

In [6]:
stocks = nyse
stocks = stocks.append(nqnm)
stocks = stocks.append(amex)
# stocks.tail()
# stocks


In [7]:
stocks = stocks[['Symbol', 'Name', 'Exchange']]
stocks.reset_index(inplace=True)
stocks = stocks.drop(columns=['index'])
stocks

,Symbol,Name,Exchange
0,A,"Agilent Technologies, Inc",NYSE
1,AA,Alcoa Corp,NYSE
2,AAN,"Aaron's, Inc",NYSE
3,AAP,"Advance Auto Parts, Inc W/I",NYSE
4,AAT,"American Assets Trust, Inc",NYSE
...,...,...,...
6276,YCBD,"cbdMD, Inc.",AMEX
6277,YCBD.PR.A,"cbdMD, Inc. 8.0% Series A Cumulative Convertib...",AMEX
6278,YUMA,"Yuma Energy, Inc",AMEX
6279,ZDGE,"Zedge, Inc",AMEX


In [8]:
# symbol = yf.Ticker('A')
# symbol.info
# symbol.quarterly_financials

In [9]:
# uncomment the following when debugging
stocks = stocks[0:5]
stocks

,Symbol,Name,Exchange
0,A,"Agilent Technologies, Inc",NYSE
1,AA,Alcoa Corp,NYSE
2,AAN,"Aaron's, Inc",NYSE
3,AAP,"Advance Auto Parts, Inc W/I",NYSE
4,AAT,"American Assets Trust, Inc",NYSE


In [16]:
def getInfo(symbol, info=True, calendar=True, quarterly_financials=True):
    connection_tries=0
    j=0
    
    # these booleans are used so that we don't get the same data again if a connection error happens afterwards
    info_bool = False
    calendar_bool = False
    quarterly_bool = False
    
    info = {}
    calendar = {}
    quarterly_financials = {}
    
    while 1:
        
        try:
            if info_bool == False:
                info = symbol.info
                info_bool = True
                
            if calendar_bool == False:
                calendar = symbol.calendar
                calendar_bool = True
                
            if quarterly_bool == False:
                quarterly_financials = symbol.quarterly_financials
                quarterly_bool = True
                
            return info, calendar, quarterly_financials
            
        except (urllib.error.HTTPError, requests.exceptions.ConnectionError) as e:
            
            if connection_tries < 1:
                print (e, "HTTP Error. Will try again in 10 seconds")
                time.sleep(10)
                connection_tries +=1
            elif connection_tries < 2:
                print (e, "HTTP Error. Will try again in 30 seconds")
                time.sleep(30)
                connection_tries +=1
            elif connection_tries < 4:
                print (e, "HTTP Error. Will try again in 5 minutes")
                time.sleep(300)
                connection_tries +=1
            elif connection_tries < 5:
                print (e, "HTTP Error.  Will try again in 1 hour")
                time.sleep(3600)
                connection_tries +=1
            else:
                connection_tries = 0
                print (e, "HTTP Error.  Giving up.")
                break
        except:
            e = sys.exc_info()[0]
            print(e)
            if j>2:
                return {}, pd.DataFrame(), pd.DataFrame()
            j+=1
            

In [27]:
i = 0
for row in stocks.itertuples():
    print (i)
    symbol = row[1]
    symbol = yf.Ticker(symbol)
    
    features = ['industry', 'sector', 'dividendRate', 'exDividendDate', 'lastSplitDate', 'lastSplitFactor']
    
    for feature in features:
        stocks = stocks.assign(feature=np.nan)
    
    
    stocks[features] = stocks[features].astype('object')
    
    info, calendar, quarterly_financials = getInfo(symbol)
    for feature in features:
        
        #try twice in case there's a fail that shouldn't happen
       
        try:
            value = info[feature]
            stocks.at[i, feature] = value
            
        except:
            stocks.at[i, feature] = np.nan
            e = sys.exc_info()[0]
            print(e, ": ", feature)
    
#     get any upcoming earnings date
    try:       
        if 'Value' in calendar.columns:
            column = 'Value'
        else:
            column = 0
        value = calendar[column]['Earnings Date']
        stocks.at[i, 'upcoming_earnings_date'] = value
    except:
        stocks.at[i, 'upcoming_earnings_date'] = np.nan
    
    #get the previous quarterly earnings date
    try:
        value = quarterly_financials.columns[0]
        stocks.at[i, 'previous_earnings_date'] = value
    except:
        stocks.at[i, 'previous_earnings_date'] = np.nan
    
    i += 1
    

0
1
2
3
4


In [28]:
stocks

,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date,feature
0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1593388800,1414972800,1398:1000,2020-11-23,2020-07-31,NaN
1,AA,Alcoa Corp,NYSE,Aluminum,Basic Materials,None,None,1477958400,1000:801,2020-10-14,2020-06-30,NaN
2,AAN,"Aaron's, Inc",NYSE,Rental & Leasing Services,Industrials,0.16,1600214400,1271376000,3:2,2020-11-02,2020-06-30,NaN
3,AAP,"Advance Auto Parts, Inc W/I",NYSE,Specialty Retail,Consumer Cyclical,1,1600300800,1127692800,3:2,2020-11-10,2020-07-11,NaN
4,AAT,"American Assets Trust, Inc",NYSE,REIT—Diversified,Real Estate,1,1599609600,None,None,2020-10-27,2020-06-30,NaN


In [63]:
stocks.to_csv('info.csv')

In [96]:
print (stocks['lastSplitDate'][0])

1414972800.0


In [19]:
pd.options.mode.chained_assignment='raise'